# Import workspace if federation is run in distributed mode

You can use any method you want to transfer exported archive to the collaborator node. If you have SSH access to the aggregator node, you might use `scp user@host:/path/to/workspace.zip workspace.zip` command. Also, you need to copy certificate authority and the collaborator key to run the collaborator process. Skip this step if you are running the federation on a single node.

In [1]:
!pip install paramiko scp

from paramiko import SSHClient, AutoAddPolicy
from scp import SCPClient
from pathlib import Path
from getpass import getpass
afqdn = # 'example.domain.com'
workspace_src = # 'path/to/workspace'
archive_name = # 'workspace.zip'
signed_cert = 'agg_to_col_two_signed_cert.zip'
key = 'cert/client/col_two.key'  
folder_name = archive_name.split('.')[0]
workspace_dst = # '/path/to/local/workspace'
src_user = # 'user'


ssh_client = SSHClient()
ssh_client.set_missing_host_key_policy(AutoAddPolicy())
ssh_client.connect(afqdn, username=src_user, password=getpass(f'Enter {src_user}@{afqdn} password:'))
scp = SCPClient(ssh_client.get_transport())
scp.get(str(Path(workspace_src) / archive_name), workspace_dst)
scp.get(str(Path(workspace_src) / signed_cert), workspace_dst)

Enter itrushki@nnlicv431.inn.intel.com password:········


In [2]:
%cd $workspace_dst
!rm -rf $folder_name
!fx workspace import --archive $archive_name
!unzip $signed_cert -d $folder_name/cert
%cd $folder_name
import os
key_dir = str(Path(key).parents[0])
os.makedirs(key_dir, exist_ok=True)
scp.get(str(Path(workspace_src) / key), key_dir)

/home/itrushkin
Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-02 14:31:47.407375: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-02 14:31:47.407405: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Workspace workspace.zip has been imported.
You may need to copy your PKI certificates to join the federation.

 ✔️ OK
Archive:  agg_to_col_two_signed_cert.zip
   creating: workspace/cert/client/
  inflating: workspace/cert/cert_chain.crt  
  inflating: workspace/cert/client/col_two.crt  
/home/itrushkin/workspace


# Choose a device

In [3]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


# Run the collaborator process

In [4]:
!fx collaborator start -n two

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-02 14:31:50.737597: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-02 14:31:50.737621: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[14:31:51] INFO     Loading DEFAULTS for section aggregator from file plan/defaults/aggregator.yaml.                                                  ]8;id=1606908711.74278-819578;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:87
           INFO     Loading DEFAULTS for section assigner from file plan/defaults/assigner.yaml.                                                      ]8;id=1606908711.7446287-324358;file:///home/itrushkin/.virtualenvs/fledge/lib

[14:31:53] INFO     Histology > X_train Shape : (1999, 3, 150, 150)                                                                       ]8;id=1606908713.3527443-149913;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:128
           INFO     Histology > y_train Shape : (1999,)                                                                                   ]8;id=1606908713.3542438-885885;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:129
           INFO     Histology > Train Samples : 1999                                                                                      ]8;id=1606908713.355639-887638;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:130
           INFO     Histology > Valid Samples : 499                                                                                       ]8;id=1606908713.3569438-530413;file:///home/itrushkin/workspace/code/histology_utils

           INFO     Received the following tasks: ['aggregated_model_validation', 'train', 'locally_tuned_model_validation']                 ]8;id=1606908715.436394-397474;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:137
/home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/task/runner_pt.py:236: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)
[14:31:57] INFO     Sending metric for task aggregated_model

train epoch: 16it [00:04,  3.36it/s]
[14:34:12] INFO     Sending metric for task train, round number 2: CrossEntropyLoss 1.379079818725586                                        ]8;id=1606908852.0919344-678210;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:324
[14:34:22] INFO     Sending metric for task locally_tuned_model_validation, round number 2: acc     0.5070140280561122                       ]8;id=1606908862.4341004-955854;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:324
           INFO     Waiting for tasks...                                                                                                     ]8;id=1606908862.7444618-389504;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:167
[14:34

[14:36:13] INFO     Sending metric for task aggregated_model_validation, round number 6: acc        0.6993987975951904                       ]8;id=1606908973.846582-453099;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:324
train epoch: 16it [00:04,  3.30it/s]
[14:36:19] INFO     Sending metric for task train, round number 6: CrossEntropyLoss 0.8828177452087402                                       ]8;id=1606908979.0248342-461709;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:324
[14:36:33] INFO     Sending metric for task locally_tuned_model_validation, round number 6: acc     0.6893787575150301                       ]8;id=1606908993.2112372-185721;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:324
       

[14:38:16] INFO     Sending metric for task aggregated_model_validation, round number 10: acc       0.751503006012024                        ]8;id=1606909096.9189773-564887;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:324
train epoch: 16it [00:04,  3.33it/s]
[14:38:22] INFO     Sending metric for task train, round number 10: CrossEntropyLoss        0.7225041389465332                               ]8;id=1606909102.0607908-23636;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:324
[14:38:32] INFO     Sending metric for task locally_tuned_model_validation, round number 10: acc    0.7154308617234469                       ]8;id=1606909112.1396825-574485;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:324
       

[14:40:19] INFO     Waiting for tasks...                                                                                                     ]8;id=1606909219.466241-237060;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:167
           INFO     Received the following tasks: ['aggregated_model_validation', 'train', 'locally_tuned_model_validation']                 ]8;id=1606909219.485715-760927;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:137
[14:40:21] INFO     Sending metric for task aggregated_model_validation, round number 14: acc       0.811623246492986                        ]8;id=1606909221.6927845-487958;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:324
train epoch: 16it [00:04,  3.31it/s]
[14:40:2

[14:42:16] INFO     Sending metric for task locally_tuned_model_validation, round number 17: acc    0.7955911823647295                       ]8;id=1606909336.7968545-454124;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:324
[14:42:19] INFO     Waiting for tasks...                                                                                                     ]8;id=1606909339.7461221-449816;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:167
           INFO     Received the following tasks: ['aggregated_model_validation', 'train', 'locally_tuned_model_validation']                 ]8;id=1606909339.7653487-37051;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/component/collaborator/collaborator.py\collaborator.py]8;;\:137
[14:42:22] INFO     Sending metric for task 